In [1]:
import os
print(f"Current working folder: {os.getcwd()}")

os.chdir('D:/My Document/Khóa Luận Tốt Nghiệp/Model Reposity/RAG_LLM_DA')
print(f"Current working folder: {os.getcwd()}")

from stages.stage_1_learn_rules_from_data.data_loader import DataLoader
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from openai_llm.llm_init import LLM_Model
from utils import load_json_data, save_json_data, load_vectorstore_db, lookup_vector_db

Current working folder: d:\My Document\Khóa Luận Tốt Nghiệp\Model Reposity\RAG_LLM_DA\notebooks
Current working folder: D:\My Document\Khóa Luận Tốt Nghiệp\Model Reposity\RAG_LLM_DA


In [2]:
llm_instance = LLM_Model()

Loading data from config/llm_config.json


In [3]:
data_dict = load_vectorstore_db(llm_instance, 'icews14')
for collection in data_dict:
    print(f"{collection}: {len(data_dict[collection]['vector_db'].get()['documents'])} documents")

Loading data from config/data_embedding.json
facts: 166718 documents
rules: 0 documents


In [26]:


filter = {"$and": [
	{"object": {"$in": ["Kashim_Shettima", 'Government_(Nigeria)']}},
	{"subject": {"$in": ["Kashim_Shettima", 'Government_(Nigeria)']}},
]
}

# filter = {"subject": "Niger"}
docs = lookup_vector_db("", filter, data_dict['facts']['vector_db'], llm_instance)
docs

[Document(metadata={'object': 'Kashim_Shettima', 'object_id': 359, 'relation': 'inv_Praise_or_endorse', 'relation_id': 237, 'subject': 'Government_(Nigeria)', 'subject_id': 9, 'timestamp': '2014-08-22', 'timestamp_id': 234}, page_content='"Government_(Nigeria) was praised or endorsed by to/with Kashim_Shettima on 2014-08-22"'),
 Document(metadata={'object': 'Kashim_Shettima', 'object_id': 359, 'relation': 'inv_Praise_or_endorse', 'relation_id': 237, 'subject': 'Government_(Nigeria)', 'subject_id': 9, 'timestamp': '2014-09-19', 'timestamp_id': 262}, page_content='"Government_(Nigeria) was praised or endorsed by to/with Kashim_Shettima on 2014-09-19"'),
 Document(metadata={'object': 'Government_(Nigeria)', 'object_id': 9, 'relation': 'Make_statement', 'relation_id': 0, 'subject': 'Kashim_Shettima', 'subject_id': 359, 'timestamp': '2014-10-10', 'timestamp_id': 283}, page_content='"Kashim_Shettima Made a statement to/with Government_(Nigeria) on 2014-10-10"'),
 Document(metadata={'object':